In [ ]:
import os
import multiprocessing
import pickle

In [7]:
class StageMatrix:
    offset = None
    window = None
    
    path_fam = None
    path_famHk= None
    path_famAlidata = None
    
    BASE = None
    struct_featureProfile = None
    struct_executionTrace = None
    struct_alignment = None
    struct_matchMatrix = None
    struct_gapSeqList = None
    struct_stageMatrix = None
    
    motif_apiparam = None
    motif_lb2seg = None
    motif_seg2lb = None
    motif_stageMatrixIndex = None
    
    def __init__(self, fam):
        self.offset = 0
        self.window = 1
        
        self.path_fam='data/family/'+ fam
        self.path_famHk = self.path_fam +'/hooklogs/'
        self.path_famAlidata = self.path_fam +'/'+fam+'_alignment.p' 
        
        self.BASE = sorted(os.listdir(self.path_famHk))[0]
        
        self.__setFeatureProfile()
        self.__setAlignment()
        self.__setMatchGap()
        if self.__checkStruct():
            self.__setStageMatrix()
            self.__setMotif()
    
    def __len__(self):
        return len(struct_stageMatrix)

    def __iter__(self):
        return iter(struct_stageMatrix)
    
    def __getitem__(self, key):
        return struct_stageMatrix[key]
    
    def __str__(self):
        return "class stageMatrix, %s, BASE = %s,\n path = %s, offset = %d, windows = %d" % (
            self.path_fam.split('/')[-1], self.BASE, self.path_fam, self.offset, self.window)
        
    # private function 
    def __setFeatureProfile(self):
        %run "0_testDef_addFeature.ipynb"
        featureProfile = dict()
        executionTrace = dict()
        
        for f in os.listdir(self.path_famHk):
            if f.endswith('.trace.hooklog'):
                fh = FeatureHooklog(self.path_famHk + f, 0)
                th = FeatureHooklog(self.path_famHk + f, 1) #設定 isAll=1 以取得所有參數與回傳值
                featureProfile[f] = fh.getHkli_noContainTS()
                executionTrace[f] = th.getHkli_containTS()
        
        self.struct_featureProfile = featureProfile
        self.struct_executionTrace = executionTrace
        print '---1 featrureProfile fin ---'
        
    def __setAlignment(self):
        %run "classes/alignment/Alignment.ipynb"
        featureProfile = self.struct_featureProfile

        print '\ttotal ',len(featureProfile.keys()), 'Execution Traces',

        if not os.path.isfile(self.path_famAlidata): 
           
            pool = multiprocessing.Pool(processes=7) #可以調整processes數量parallel computing

            ali = dict()
            for hk in featureProfile.keys():
                fp_base = featureProfile[self.BASE]
                fp_hk = featureProfile[hk]
                ali[hk] = pool.apply_async(pairwise_NW, ( fp_base, fp_hk, 2, -1, -3, 1, self.offset, self.window))
            pool.close()
            pool.join()

            alignment= dict()
            for t in ali:
                alignment[t] = ali[t].get()[2]
            pickle.dump(alignment, open(self.path_famAlidata, "wb"))

        else:
            alignment = pickle.load(open(self.path_famAlidata,'rb')) 
            print '(read alignment pickle)'
            
        self.struct_alignment = alignment

        print '\tall pairwise alignment done.'
        print '---2 alignment fin ---'
  
    def __setMatchGap(self):
        len_BASE = len(self.struct_featureProfile[self.BASE])
        
        matchMatrix={i+1:[] for i in xrange(len_BASE)} # index start from 1 to |PB|
        gapSeqList={i:[] for i in xrange(len_BASE +1)} # start is gap, end is not gap
        
        for i,hk in enumerate(self.struct_alignment):
            isMeet1 = 0
            len_hk = len(self.struct_featureProfile[hk])
            ali_hk = self.struct_alignment[hk]
            
            if len_hk > len_BASE :
                for ii,((i1,t1),(i2,t2)) in enumerate(ali_hk):
                    #判斷是否有在PB[0]之前就插入
                    if not isMeet1 and i1 == -1:
                        strBuf_list = [t2]
                        while ali_hk[ii+1][0][0] == -1:
                            strBuf_list.append(ali_hk[ii+1][1][1])
                            ii += 1
                        gapSeqList[0].append((hk, strBuf_list))
                        isMeet1 = 1
                    else:
                        isMeet1 = 1

                    #判斷PB[0]之後的情形
                    if t1=='=' and t2!='=' and isMeet1 and ali_hk[ii-1][0][0]!=-1:
                        lastIndex = ali_hk[ii-1][0][0]
                        strBuf_list = [t2]
                        try:
                            while ali_hk[ii+1][0][0] == -1:
                                strBuf_list.append(ali_hk[ii+1][1][1])
                                ii += 1
                        except:
                            pass                
                        gapSeqList[lastIndex].append((hk, strBuf_list))
                    elif t2=='=' and t1!='=':
                        matchMatrix[i1].append((hk,'gap','=')) 
                    elif t1!='=' and t2!='=' and t1!=t2:
                        matchMatrix[i1].append((hk,'mismatch',t2))
                    elif t2!='=' and t1!='=' and t1==t2:
                        matchMatrix[i1].append((hk,'match',t2))

            elif len_hk <= len_BASE :
                for ii,((i1,t1),(i2,t2)) in enumerate(ali_hk):
                    #判斷是否有在PB[0]之前就插入
                    if not isMeet1 and i2 == -1:
                        strBuf_list = [t1]
                        while ali_hk[ii+1][1][0] == -1:
                            strBuf_list.append(ali_hk[ii+1][0][1])
                            ii += 1
                        gapSeqList[0].append((hk, strBuf_list))
                        isMeet1 = 1
                    else:
                        isMeet1 = 1

                    #判斷PB[0]之後的情形
                    if t2=='=' and t1!='=' and isMeet1 and ali_hk[ii-1][1][0]!=-1:
                        lastIndex = ali_hk[ii-1][1][0]
                        strBuf_list = [t1]
                        try:
                            while ali_hk[ii+1][1][0] == -1:
                                strBuf_list.append(ali_hk[ii+1][0][1])
                                ii += 1
                        except:
                            pass
                        gapSeqList[lastIndex].append((hk, strBuf_list))
                    elif t1=='=' and t2!='=':
                        matchMatrix[i2].append((hk,'gap','=')) 
                    elif t2!='=' and t1!='=' and t1!=t2:
                        matchMatrix[i2].append((hk,'mismatch',t1))
                    elif t2!='=' and t1!='=' and t1==t2:
                        matchMatrix[i2].append((hk,'match',t1))
                        
        self.struct_matchMatrix = matchMatrix
        self.struct_gapSeqList = gapSeqList
        print '---3 match& gap fin ---'
        
    def __setStageMatrix(self):
        featureProfile = self.struct_featureProfile
        gapSeqList = self.struct_gapSeqList
        matchMatrix = self.struct_matchMatrix
        
        stageMatrix = {k:[] for k in featureProfile}
        
        for i,k in enumerate(featureProfile):
            strBuf_list = []
            for ii in range(len(featureProfile[self.BASE])+1):
                if ii == 0:
                    #有在gapSeqList的話就加入，k對應的list，否則加入gap
                    if len(gapSeqList[0]) > 0:
                        gapli = filter(lambda (hk,api):hk==k, gapSeqList[0])
                        if gapli == list():
                            stageMatrix[k].append(['=']) 
                        else:
                            print gapli
                            stageMatrix[k].append(gapli[0][1])
                            
                else:
                    a = [api for hk, stat, api in matchMatrix[ii] if k==hk][0]
                    strBuf_list.append(a)

                    if len(gapSeqList[ii]) > 0:
                        stageMatrix[k].append(strBuf_list)
                        strBuf_list = []       

                        gapli = filter(lambda (hk,api):hk==k, gapSeqList[ii])
                        if gapli == list():
                            stageMatrix[k].append(['=']) 
                        else:
                            stageMatrix[k].append(gapli[0][1])
                            
                    else:
                        if ii < len(matchMatrix):
                            curColStat_list = [stat for kin, stat, api in matchMatrix[ii]]
                            nextColStat_list = [stat for kin, stat, api in matchMatrix[ii+1]]
                            if curColStat_list != nextColStat_list:
                                stageMatrix[k].append(strBuf_list)
                                strBuf_list = []
                        elif ii == len(matchMatrix): # 若爲最後直接 append 進去
                            stageMatrix[k].append(strBuf_list)
                            strBuf_list = []
                            
                                
        
        self.struct_stageMatrix = stageMatrix
        print '---4 stageMatrix fin ---'
    
    def __setMotif(self):
        %run "classes/alignment/OrderedSet.ipynb"
        
        stageMatrix = self.struct_stageMatrix
        BASE = self.BASE

        segment_set = OrderedSet()
        for i in range(len(stageMatrix[BASE])): 
            for hk in stageMatrix:
                segment_set.add(tuple(stageMatrix[hk][i]))
        print '\ttotal segment #: ',len(segment_set) 

        #apiparam set
        apiparam_set = set()
        for hk in stageMatrix: 
            for seg in stageMatrix[hk]:
                for a in seg:
                    apiparam_set.add(a)

        #label each segment (motif)
        lb2seg_dict = dict()
        for i in range(len(segment_set)):
            lb2seg_dict['M'+str(i+1)]= tuple(list(segment_set)[i])
        seg2lb_dict = dict()
        for k in lb2seg_dict:
            seg2lb_dict[lb2seg_dict[k]] = k
            
        #set stageMatrixIndex
        #記錄各 hk 中每個stage對應到的motif, 在hk的 "起始&結束 index, 用於API deduplication
        stageMatrixIndex = dict()
        for hklg in stageMatrix:
            idxtuple_list = []
            last_start = 0
            last_end = 0
            for motif in stageMatrix[hklg]:
                if motif == ['=']:
                    idxtuple_list.append((-1, -1))
                    continue
                last_end = last_start + len(motif) - 1
                idxtuple_list.append((last_start, last_end))
                last_start = last_end + 1
            stageMatrixIndex[hklg] = idxtuple_list
    
        self.motif_apiparam = apiparam_set
        self.motif_lb2seg = lb2seg_dict
        self.motif_seg2lb = seg2lb_dict
        self.motif_stageMatrixIndex = stageMatrixIndex
        print '---5 setMotif fin ---'
    
    
    def __checkStruct(self): #recheck data in structures are the same
        alignment = self.struct_alignment
        gapSeqList = self.struct_gapSeqList
        matchMatrix = self.struct_matchMatrix
        
        for k in alignment:
            len_featureK = len(self.struct_featureProfile[k])
            totalLen = 0

            for i in gapSeqList:
                for kin, l in gapSeqList[i]:
                    if k == kin:
                        totalLen += len(l)
                        break
            for i in matchMatrix:
                for kin, stat, api in matchMatrix[i]:
                    if k == kin and stat != 'gap':
                        totalLen += 1
                        break
            if len_featureK != totalLen: 
                print k, 'structure has error!'
                return False
        return True
    
        
    # public function
    def getFamPath(self):
        return self.path_fam
    
    def getBASE(self):
        return self.BASE
    
    def getFeatureProfile(self): 
        return self.struct_featureProfile
    
    def getExecutionTrace(self): 
        return self.struct_executionTrace
    
    def getAlignment(self):
        return self.struct_alignment

    def getMatchMatrix(self):
        return self.struct_matchMatrix
    
    def getGapSeqList(self):
        return self.struct_gapSeqList
    
    def getStageMatrix(self):
        return self.struct_stageMatrix
    
    def getStageMatrixIndex(self):
        return self.motif_stageMatrixIndex
    
    def getMoti_apiPar(self):
        return self.motif_apiparam
    
    def getMoti_lb2seg(self):
        return self.motif_lb2seg
    
    def getMoti_seg2lb(self):
        return self.motif_seg2lb